In [82]:

from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from PyQt5.QtWidgets import  QApplication, QMainWindow, QShortcut, QFileDialog , QSplitter , QFrame , QSlider
from scipy.signal import spectrogram
from scipy.signal import resample
import sys
from PyQt5.QtGui import QIcon, QKeySequence
from mainwindow import Ui_MainWindow  
from pyqtgraph import PlotWidget, ROI

import numpy as np
import pandas as pd
from scipy.io import wavfile
import pyqtgraph as pg
from scipy.fftpack import rfft, rfftfreq, irfft , fft , fftfreq
from PyQt5.QtCore import pyqtSlot
import sounddevice as sd
import librosa
import os


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [83]:
folder_path = "C:/Users/Sara/Desktop/sara_voice_code_access/voices"
files = os.listdir(folder_path)

df = pd.DataFrame(columns = ['max_pitch', 'avg_pitch', 'variance_pitch', 'harmonic',
       'harmonic_variance', 'percussive', 'percussive_variance', 'Chroma_cens',
       'Chroma_cens_var', 'chroma_cqt_mean', 'chroma_cqt_variance',
       'chroma_stft_mean', 'chroma_stft_var', 'mfcc_delta_mean',
       'mfcc_delta_variance', 'Mfccs', 'Mfccs_variance', 'Contrast',
       'Contrast_var', 'poly_features_mean', 'poly_features_variance',
       'Rolloff', 'Rolloff_var', 'Zrate', 'Zrate_var', 'Cent', 'Cent_var',
       'tonnetz_mean', 'tonnetz_var', 'spec_bw_mean', 'spec_bw_var',
       'rmse_mean', 'rmse_var', 'melspec_mean', 'melspec_var', 'max_lpc',
       'mean_lpc', 'var_lpc' , 'user' , 'sentense'])
l = 0
for file in files:
    file_path = os.path.join(folder_path, file)
    samples , sr = librosa.load(file_path)
    # print(file_path)
    S = np.abs(librosa.stft(samples))
    pitches,magnitudes = librosa.core.piptrack(y = samples ,sr = sr)
    
    max_pitch = np.max(pitches)
    avg_pitch = np.mean(pitches)
    variance_pitch = np.var(pitches)
                    
    y_harmonic, y_percussive = librosa.effects.hpss(samples)
    harmonic = np.mean(y_harmonic)
    harmonic_variance = np.var(y_harmonic)
    percussive = np.mean(y_percussive)
    percussive_variance = np.var(y_percussive)
                    
    chroma=librosa.feature.chroma_cens(y=y_harmonic, sr=sr)
    Chroma_cens = np.mean(chroma)
    Chroma_cens_var = np.var(chroma)
    chroma_stft =librosa.feature.chroma_stft(y=y_harmonic, sr=sr)
    chroma_stft_mean = np.mean(chroma_stft)
    chroma_stft_var = np.var(chroma_stft)
                    
    chroma_cqt =librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)
    chroma_cqt_mean = np.mean(chroma_cqt)
    chroma_cqt_variance = np.var(chroma_cqt)
                    
    poly_features = librosa.feature.poly_features(S=S, sr=sr)
    poly_features_mean = np.mean(poly_features)
    poly_features_variance = np.var(poly_features)
                    
    mfccs = librosa.feature.mfcc(y=y_harmonic, sr=sr)
    Mfccs = np.mean(mfccs)
    
    Mfccs_variance = np.var(mfccs)
    delta = librosa.feature.delta(mfccs)
    mfcc_delta_mean = np.mean(delta)
    mfcc_delta_variance = np.var(delta)
                    
    melspectrogram = librosa.feature.melspectrogram(y=samples, sr=sr)
    melspec_mean = np.mean(melspectrogram)
    melspec_var = np.var(melspectrogram)
                    
    contrast=librosa.feature.spectral_contrast(y=y_harmonic,sr=sr)
    Contrast = np.mean(contrast)
    Contrast_var = np.var(contrast)
                    
    rolloff = librosa.feature.spectral_rolloff(y=samples, sr=sr)      
    Rolloff = np.mean(rolloff)
    Rolloff_var = np.var(rolloff)
                    
    zrate=librosa.feature.zero_crossing_rate(y_harmonic)
    Zrate = np.mean(zrate)
    Zrate_var = np.var(zrate)
                    
    cent = librosa.feature.spectral_centroid(y=samples, sr=sr)
    Cent = np.mean(cent)
    Cent_var = np.var(cent)
                    
    tonnetz = librosa.feature.tonnetz(y=samples, sr=sr)
    tonnetz_mean = np.mean(tonnetz)
    tonnetz_var = np.var(tonnetz)
                    
    spec_bw = librosa.feature.spectral_bandwidth(y=samples, sr=sr)
    spec_bw_mean = np.mean(spec_bw)
    spec_bw_var = np.var(spec_bw)
                                         
    rmse=librosa.feature.rms(y=samples)[0]
    rmse_mean = np.mean(rmse)
    rmse_var = np.var(rmse)
                                         
    melspectrogram = librosa.feature.melspectrogram(y=samples, sr=sr)
    melspec_mean = np.mean(melspectrogram)
    melspec_var = np.var(melspectrogram)
    
    lpc=librosa.lpc(samples, order=16)
    max_lpc=np.max(lpc)
    mean_lpc=np.mean(lpc)
    var_lpc=np.var(lpc)
    
    target_user , target_sentense = os.path.splitext(file)[0].split('-')
    print(target_sentense , target_user)
    
    data = list([max_pitch, avg_pitch,variance_pitch,harmonic,harmonic_variance,percussive, percussive_variance, Chroma_cens,Chroma_cens_var, chroma_cqt_mean, chroma_cqt_variance,chroma_stft_mean, chroma_stft_var, mfcc_delta_mean,mfcc_delta_variance, Mfccs, Mfccs_variance, Contrast,Contrast_var, poly_features_mean, poly_features_variance,Rolloff, Rolloff_var, Zrate, Zrate_var, Cent, Cent_var,tonnetz_mean,tonnetz_var,spec_bw_mean,spec_bw_var,rmse_mean, rmse_var, melspec_mean, melspec_var, max_lpc,mean_lpc,var_lpc , target_user , target_sentense])
    df.loc[l] = data
    l+=1
    

openTheDoor sara


In [84]:
df.to_csv("Team_features.csv",index=False)


In [85]:
df_from_csv = pd.read_csv("Team_features.csv")
last_three_columns = df_from_csv.iloc[:, -5:]
print(last_three_columns.head())

   max_lpc  mean_lpc   var_lpc  user     sentense
0      1.0  0.001005  0.344704  sara  openTheDoor


In [86]:
df

,max_pitch,avg_pitch,variance_pitch,harmonic,harmonic_variance,percussive,percussive_variance,Chroma_cens,Chroma_cens_var,chroma_cqt_mean,...,spec_bw_var,rmse_mean,rmse_var,melspec_mean,melspec_var,max_lpc,mean_lpc,var_lpc,user,sentense
0,3808.014648,16.288244,25154.392578,-4.428989e-07,0.000545,-0.000002,0.000676,0.251823,0.019919,0.484787,...,277005.077189,0.017356,0.001398,0.404011,37.366776,1.0,0.001005,0.344704,sara,openTheDoor


In [87]:
features = df.drop(['user' , 'sentense'] , axis=1)
target_sentense = pd.DataFrame(df["user"]) 
target_user = pd.DataFrame(df["sentense"]) 
print(features.head())
print(target_sentense.head())
print(target_user.head())


     max_pitch  avg_pitch  variance_pitch      harmonic  harmonic_variance  \
0  3808.014648  16.288244    25154.392578 -4.428989e-07           0.000545   

   percussive  percussive_variance  Chroma_cens  Chroma_cens_var  \
0   -0.000002             0.000676     0.251823         0.019919   

   chroma_cqt_mean  ...  tonnetz_var  spec_bw_mean    spec_bw_var  rmse_mean  \
0         0.484787  ...     0.005325   2265.635973  277005.077189   0.017356   

   rmse_var  melspec_mean  melspec_var  max_lpc  mean_lpc   var_lpc  
0  0.001398      0.404011    37.366776      1.0  0.001005  0.344704  

[1 rows x 38 columns]
   user
0  sara
      sentense
0  openTheDoor


In [88]:
X_train_sentense, X_test_sentense, y_train_sentense, y_test_sentense = train_test_split(features, target_sentense, test_size=0.2, random_state=42)

# Split the data into training and testing sets for user prediction
X_train_user, X_test_user, y_train_user, y_test_user = train_test_split(features, target_user, test_size=0.2, random_state=42)

# Initialize and train the Random Forest models for sentense and user prediction
rf_sentense = RandomForestClassifier(n_estimators=100, random_state=42)
rf_sentense.fit(X_train_sentense, y_train_sentense)

rf_user = RandomForestClassifier(n_estimators=100, random_state=42)
rf_user.fit(X_train_user, y_train_user)

# Make predictions on the test set for sentense and user
predictions_sentense = rf_sentense.predict(X_test_sentense)
predictions_user = rf_user.predict(X_test_user)

# Evaluate the accuracy of the models
accuracy_sentense = accuracy_score(y_test_sentense, predictions_sentense)
accuracy_user = accuracy_score(y_test_user, predictions_user)
print(accuracy_sentense)
print(accuracy_user)

ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.